In [90]:
%reload_ext autoreload
%autoreload 2
import ast
import pickle
import itertools
from collections import Counter
from tqdm import tqdm
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import swifter

import sys
sys.path.append('..')
from analysis.generate_cluster_information_file import load, extract_all_information_query, to_df_query
from common.image_processing import pixel_intensity_histogram


from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
import xgboost
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [81]:
train_X = pd.read_csv('train_X.csv', dtype='float')
train_y = pd.read_csv('train_y.csv', dtype='float')

na_indices = train_X[train_X.isna().any(axis=1)].index.values
train_X.drop(na_indices, inplace=True)
train_y.drop(na_indices, inplace=True)

print(train_X.shape)
train_X.head(10)

(9600, 21)


,cluster_num,background_threshold,cluster_num_intensities_avg,cluster_num_intensities_max,cluster_num_intensities_min,cluster_num_intensities_std,cluster_peak_intensities_avg,cluster_peak_intensities_max,cluster_peak_intensities_min,cluster_peak_intensities_std,...,cluster_sizes_std,cluster_sizes_max,cluster_sizes_min,cluster_sizes_sum,cluster_x_avg,cluster_y_avg,cluster_dist_min,cluster_dist_max,largest_clusters_distance,brightest_clusters_distance
0,50.0,0.0,13.844046,1.007997e-311,1.008275e-311,1.008275e-311,14.920000,255.0,1.0,49.370777,...,46.114679,225.0,1.0,1779.0,513.460000,475.320000,19.235384,1251.265360,418.150691,418.150691
1,104.0,0.0,11.164915,1.007929e-311,1.007929e-311,1.007929e-311,11.375000,255.0,1.0,36.751030,...,36.295014,179.0,1.0,2926.0,536.201923,473.798077,17.204651,1223.943218,537.729486,605.561723
2,71.0,1.0,12.496173,4.649718e+02,2.730000e+02,1.000000e+00,11.563380,255.0,2.0,34.161890,...,49.933755,273.0,1.0,2426.0,464.971831,513.746479,24.041631,1235.969255,758.641549,841.503417
3,49.0,2.0,13.074356,4.172857e+02,6.350000e+02,1.000000e+00,22.612245,255.0,3.0,51.612297,...,107.127941,635.0,1.0,2628.0,417.285714,564.571429,13.601471,1243.556995,798.068919,257.670332
4,169.0,0.0,6.639654,4.622249e+02,1.610000e+02,1.000000e+00,6.621302,255.0,1.0,26.394791,...,29.501191,161.0,1.0,3469.0,462.224852,509.230769,2.000000,1319.351735,1064.421439,137.116739
5,51.0,1.0,17.443819,5.909216e+02,2.360000e+02,1.000000e+00,26.921569,255.0,2.0,64.565772,...,58.244074,236.0,1.0,2432.0,590.921569,492.352941,21.954498,1135.750853,344.653449,337.831319
6,2216.0,191.0,4.407226,4.335695e+02,1.512920e+05,1.000000e+00,196.498195,254.0,192.0,6.801996,...,3238.075401,151292.0,1.0,192580.0,433.569495,468.828520,2.000000,1187.283454,646.561675,366.006831
7,66.0,0.0,13.259619,4.502576e+02,2.630000e+02,1.000000e+00,15.045455,255.0,1.0,46.085044,...,45.954363,263.0,1.0,1988.0,450.257576,427.469697,8.062258,1153.087161,267.029961,267.029961
8,73732.0,9.0,1.713660,4.986155e+02,6.200000e+01,1.000000e+00,17.661884,255.0,10.0,5.843887,...,2.440176,62.0,1.0,177887.0,498.615499,497.128208,8.062258,1153.087161,1.000000,1.000000
9,35.0,1.0,14.468375,5.407714e+02,2.240000e+02,1.000000e+00,14.171429,255.0,2.0,44.876011,...,43.131692,224.0,1.0,1120.0,540.771429,514.942857,14.317821,1216.550040,444.668416,444.668416


In [94]:
def train_with_features(X, features, rf=True, svm=True, xgb=True, feature_selection=True):
    assert len(features) > 0
    assert type(features[0]) == str
    
    if features != 'all':
        X = X[features]
    else:
        X = X
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(train_y)
    
    if rf:
        print("Training Random Forest..")
        randomforest = RandomForestRegressor(max_depth=6, random_state=0)
        score_mi = cross_val_score(randomforest, X_scaled, np.ravel(y_scaled), cv=5, scoring=make_scorer(mean_absolute_error))
        print("Mean: {}, Std Dev: {}".format(score_mi.mean(), np.std(score_mi)))
        
    if svm:
        print("Training SVM..")
        randomforest = SVR(kernel='rbf')
        score_mi = cross_val_score(randomforest, X_scaled, np.ravel(y_scaled), cv=5, scoring=make_scorer(mean_absolute_error))
        print("Mean: {}, Std Dev: {}".format(score_mi.mean(), np.std(score_mi)))
        
    if xgb:
        print("Training XGBoost..")
        model = xgboost.XGBRegressor(objective="reg:squarederror", learning_rate =0.01, booster='gbtree', n_estimators=1000, max_depth=6, gamma=0.5, subsample=0.7, colsample_bytree=1.0, nthread=-1, verbosity=1)
        score_mi = cross_val_score(model, X_scaled, np.ravel(y_scaled), cv=5, scoring=make_scorer(mean_absolute_error))
        print("Mean: {}, Std Dev: {}".format(score_mi.mean(), np.std(score_mi)))

In [42]:
train_with_features(train_X, ['cluster_num'])

Mean: 0.6488447391468939, Std Dev: 0.010385121122722645


### Train with all features

In [53]:
train_with_features(train_X, 'all')

Mean: 0.46932710072849765, Std Dev: 0.007760576881457447


### Train with selected features
We select features according to the analysis in feature_selection.ipynb

In [77]:
filtered_X = train_X.drop(columns=['cluster_sizes_min', 'cluster_num_intensities_min', 'cluster_num_intensities_max', 'cluster_num_intensities_std'])
train_with_features(filtered_X, 'all')

Training XGBoost..
Mean: 0.4227118606684431, Std Dev: 0.005237206826800358


### Train without "two-largest" distance features

In [72]:
filtered_X = train_X.drop(columns=['largest_clusters_distance', 'brightest_clusters_distance'])
train_with_features(filtered_X, 'all', rf=False, svm=False)

Training XGBoost..
Mean: 0.4184656123144056, Std Dev: 0.004583597701546552


### Train any distance features

In [75]:
filtered_X = train_X.drop(columns=['cluster_dist_min', 'cluster_dist_max'])
train_with_features(filtered_X, 'all', rf=False, svm=False)

Training XGBoost..
Mean: 0.4199283815823409, Std Dev: 0.0048476628156421285


### LASSO
feature selection

In [97]:
lasso_features = ['brightest_clusters_distance', 'largest_clusters_distance', 'cluster_dist_min', 'cluster_sizes_min', 'background_threshold', 'cluster_peak_intensities_std', 'cluster_peak_intensities_min', 'cluster_peak_intensities_max']
train_with_features(train_X, lasso_features, rf=False, svm=False, xgb=True)

Training XGBoost..
Mean: 0.5085966864996017, Std Dev: 0.01275082607752883
